In [ ]:
# standard imports
import numpy as np
import xarray as xr
import rioxarray
import matplotlib.pyplot as plt

# load up the saved dataset
import pickle
# with open('/Users/zachkeskinen/Documents/spicy-snow/data/10_img_dB.pkl', 'rb') as f:
#     ds = pickle.load(f)

with open('/Users/zachkeskinen/Documents/spicy-snow/tests/test_data/2_img_ds', 'rb') as f:
    ds = pickle.load(f)

# add module to path so python can import it. This is the directory with 
# the __init__.py file in it to let python know this is a module.
import sys
sys.path.append('/Users/zachkeskinen/Documents/spicy-snow/spicy_snow')

# import the function to test
from processing.snow_index import calc_delta_VV

# ds = ds.isel(time = slice(1, 3))

In [ ]:
da1, da2, da3 = ds['s1'].sel(band = 'VV')

In [ ]:
real2_1_diff = da2 - da1

In [ ]:
ds1 = calc_delta_VV(ds)

In [ ]:
real2_1_diff

In [ ]:
ds1['s1'].sel(band = 'deltaVV').isel(time = 1)

In [ ]:
ds1['s1'].sel(band = 'deltaVV').isel(time = 6).plot(vmin = -1, vmax = 1)

In [ ]:
def add_band(dataset, dataArray, band_name):
    dataArray = dataArray.reset_coords(names = ['flight_dir', 'platform', 'relative_orbit'], drop = True)
    dataArray['band'] = band_name

    if band_name not in dataset.band:
        com_da = xr.concat([dataset['s1'], dataArray], dim = 'band')

        vars = list(dataset.data_vars)
        vars.remove('s1')

        dataset = xr.merge([dataset[vars], com_da])
    else:
        dataset['s1'].loc[dict(band = band_name, time = dataArray.time)] = dataArray

    return dataset

In [ ]:
# get all unique relative orbits
orbits = np.unique(ds['relative_orbit'].values)

vars = list(ds.data_vars)
vars.remove('s1')

for orbit in orbits:
    diff = ds['s1'].sel(time = ds.relative_orbit == orbit, band = 'VV').diff(dim = 'time')
    ds = add_band(ds, diff, 'deltaVV')

In [ ]:
ds['s1'].sel(band = 'deltaVV').isel(time = 8)

In [ ]:
ds['s1'].loc[dict(band = 'deltaVV', time = diff.time)] = diff

In [ ]:
ds

In [ ]:
add_band(ds, diff, 'deltaVV')

In [ ]:
com_da

In [ ]:
ds

In [ ]:
diff['band'] = 'deltaVV'

In [ ]:
xr.concat([ds['s1'], diff], dim = 'band')

In [ ]:
def add_band(dataset, dataArray, name):
    dataArray = dataArray.reset_coords(names = ['flight_dir', 'platform', 'relative_orbit'], drop = True)
    dataArray['band'] = name
    com_da = xr.concat([dataset['s1'], dataArray], dim = 'band')

    vars = list(dataset.data_vars)
    vars.remove('s1')
    dataset = xr.merge([dataset[vars], com_da])

    return dataset

In [ ]:
ds2 = add_band(ds, diff, 'deltaVV')

In [ ]:
ds2.sel(band = 'deltaVV')['s1'].isel(time = 3).plot(vmin = -1, vmax=1)

In [ ]:
diff = diff.reset_coords(names = ['flight_dir', 'platform', 'relative_orbit'], drop = True)
com_da = xr.concat([ds['s1'], diff], dim = 'band')

In [ ]:
vars = list(ds.data_vars)
vars.remove('s1')

a = list(ds.data_vars)
a.remove('s1')
ds = xr.merge([ds[a], com_da])


In [ ]:
ds

In [ ]:
vv1 = ds['s1'].sel(time = ds.relative_orbit == 20, band = 'VV').isel(time = 0)
vv2 = ds['s1'].sel(time = ds.relative_orbit == 20, band = 'VV').isel(time = 1)

In [ ]:
vv1.plot()

In [ ]:
vv2.plot()

In [ ]:
(vv2 - vv1).plot(vmax = 1, vmin = -1)

In [ ]:
diff.isel(time = 0).plot(vmax = 1, vmin = -1)